# RL-i treenimine Cartpole tasakaalustamiseks

See märkmik on osa [AI algajatele mõeldud õppekavast](http://aka.ms/ai-beginners). See on inspireeritud [ametlikust PyTorch juhendist](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) ja [sellest Cartpole PyTorch implementatsioonist](https://github.com/yc930401/Actor-Critic-pytorch).

Selles näites kasutame RL-i, et treenida mudelit tasakaalustama posti kärul, mis saab liikuda vasakule ja paremale horisontaalsel skaalal. Simulatsiooni jaoks kasutame [OpenAI Gym](https://www.gymlibrary.ml/) keskkonda.

> **Märkus**: Selle õppetunni koodi saab käivitada lokaalselt (nt Visual Studio Code'is), sel juhul avaneb simulatsioon uues aknas. Kui koodi käitatakse veebis, võib olla vaja teha koodis mõningaid muudatusi, nagu kirjeldatud [siin](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Alustame veendumaks, et Gym on installitud:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Nüüd loome CartPole'i keskkonna ja vaatame, kuidas sellega töötada. Keskkonnal on järgmised omadused:

* **Tegevusruum** on võimalik tegevuste kogum, mida saame simuleerimise igal sammul teha
* **Vaatlusruum** on vaatluste ruum, mida saame teha


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Vaatame, kuidas simulatsioon töötab. Järgmine tsükkel käivitab simulatsiooni, kuni `env.step` ei tagasta lõpetamise lippu `done`. Me valime juhuslikult tegevusi, kasutades `env.action_space.sample()`, mis tähendab, et eksperiment ebaõnnestub tõenäoliselt väga kiiresti (CartPole keskkond lõpetab, kui CartPole kiirus, asukoht või nurk ületavad teatud piirid).

> Simulatsioon avaneb uues aknas. Koodi saab mitu korda käivitada ja vaadata, kuidas see käitub.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Sa märkad, et vaatlused sisaldavad 4 numbrit. Need on:
- Käru asukoht
- Käru kiirus
- Posti nurk
- Posti pöörlemiskiirus

`rew` on tasu, mida saame igal sammul. CartPole'i keskkonnas antakse iga simulatsioonisammu eest 1 punkt ja eesmärk on maksimeerida kogutasu, st aeg, mille jooksul CartPole suudab tasakaalu hoida ilma ümber kukkumata.

Tugevdusõppe käigus on meie eesmärk treenida **poliitikat** $\pi$, mis iga oleku $s$ puhul ütleb, millise tegevuse $a$ peaksime valima, seega sisuliselt $a = \pi(s)$.

Kui soovid tõenäosuslikku lahendust, võid mõelda poliitikast kui tegevuste tõenäosuste komplektist, st $\pi(a|s)$ tähendaks tõenäosust, et peaksime olekus $s$ valima tegevuse $a$.

## Poliitika gradientmeetod

Lihtsaimas RL algoritmis, mida nimetatakse **poliitika gradientiks**, treenime närvivõrku ennustama järgmist tegevust.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Me treenime võrku, tehes palju katseid ja uuendades oma võrku pärast iga katset. Määratleme funktsiooni, mis viib läbi katse ja tagastab tulemused (nn **jälje**) - kõik olekud, tegevused (ja nende soovitatud tõenäosused) ning tasud:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Saad käivitada ühe episoodi treenimata võrguga ja jälgida, et kogutasu (ehk episoodi pikkus) on väga madal:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Poliitikagradientide algoritmi üks keerulisi aspekte on **diskonteeritud tasude** kasutamine. Idee seisneb selles, et arvutame mängu iga sammu kogutasude vektori ja selle protsessi käigus diskonteerime varased tasud koefitsiendiga $gamma$. Samuti normaliseerime saadud vektori, kuna kasutame seda kaalu mõjutamiseks meie treeningus:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Nüüd alustame tegelikku treeningut! Me viime läbi 300 episoodi ja igas episoodis teeme järgmist:

1. Käivitame eksperimendi ja kogume jälje
1. Arvutame erinevuse (`gradients`) võetud tegevuste ja ennustatud tõenäosuste vahel. Mida väiksem on erinevus, seda kindlamad oleme, et oleme teinud õige tegevuse.
1. Arvutame diskonteeritud tasud ja korrutame gradientid diskonteeritud tasudega - see tagab, et sammud, millel on suuremad tasud, avaldavad lõpptulemusele suuremat mõju kui madalama tasuga sammud.
1. Meie närvivõrgu oodatavad sihttegevused võetakse osaliselt jooksu ajal ennustatud tõenäosustest ja osaliselt arvutatud gradientidest. Kasutame `alpha` parameetrit, et määrata, millises ulatuses gradientid ja tasud arvesse võetakse - seda nimetatakse *õppemääraks* tugevdamise algoritmis.
1. Lõpuks treenime oma võrku olekute ja oodatavate tegevuste põhjal ning kordame protsessi.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Nüüd käivitame episoodi koos renderdamisega, et tulemust näha:


In [ ]:
_ = run_episode(render=True)

Loodetavasti näete, et post suudab nüüd üsna hästi tasakaalu hoida!

## Näitleja-Kriitik mudel

Näitleja-Kriitik mudel on poliitika gradientide edasiarendus, kus me loome tehisnärvivõrgu, et õppida nii poliitikat kui ka hinnangulisi tasusid. Võrgul on kaks väljundit (või võite seda vaadelda kui kahte eraldi võrku):
* **Näitleja** soovitab tegevuse, mida ette võtta, andes meile oleku tõenäosusjaotuse, nagu poliitika gradientide mudelis.
* **Kriitik** hindab, milline tasu võiks nendest tegevustest tulla. See tagastab tuleviku koguhinnangulised tasud antud olekus.

Määratleme sellise mudeli:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Me peaksime veidi muutma oma `discounted_rewards` ja `run_episode` funktsioone:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Nüüd käivitame peamise treeningtsükli. Kasutame käsitsi võrgu treenimise protsessi, arvutades sobivad kaotusfunktsioonid ja uuendades võrgu parameetreid:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

Lõpuks sulgeme keskkonna.


In [ ]:
env.close()

## Peamine mõte

Selles demos tutvusime kahe tugeva õppimise algoritmiga: lihtsa poliitika gradienti ja keerukama näitleja-kriitikuga. Nagu näha, töötavad need algoritmid abstraktsete oleku, tegevuse ja tasu mõistetega – seega saab neid rakendada väga erinevates keskkondades.

Tugevdatud õppimine võimaldab meil õppida parimat strateegiat probleemi lahendamiseks, tuginedes ainult lõplikule tasule. See, et me ei vaja märgistatud andmekogumeid, võimaldab meil simuleerida olukordi korduvalt, et oma mudeleid optimeerida. Siiski on RL-is endiselt palju väljakutseid, mida saate avastada, kui otsustate süveneda sellesse põnevasse tehisintellekti valdkonda.



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
